In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import sys
sys.path.append('../code')
import compute_indicators as ci

In [4]:
import numpy as np
import pandas as pd
import talib as ta
from tqdm.notebook import tqdm

In [5]:
import alpaca_trade_api as tradeapi

key_id = None
secret_key = None
with open('../files/private/alpakey') as key_file:
    keys = key_file.readlines()
    key_id_control = keys[0].strip()
    secret_key_control = keys[1].strip()
    key_id_market = keys[2].strip()
    secret_key_market = keys[3].strip()
    key_id_no_market = keys[4].strip()
    secret_key_no_market = keys[5].strip()
    
api_url = "https://paper-api.alpaca.markets"

alpaca_control = tradeapi.REST(key_id_control, secret_key_control, api_url, api_version='v2')
alpaca_market = tradeapi.REST(key_id_market, secret_key_market, api_url, api_version='v2')
alpaca_no_market = tradeapi.REST(key_id_no_market, secret_key_no_market, api_url, api_version='v2')

In [6]:
import json

symbols = []
with open('../files/public/sp_100.json') as top_symbols:
    symbols = json.load(top_symbols)

In [7]:
columns_market = json.load(open('../models/columns.json', 'rb'))

columns_no_market = list(filter(lambda x: 'market' not in x, columns_market))

In [8]:
from keras.models import load_model

model_market = load_model('../models/daily_market.h5')
model_no_market = load_model('../models/daily.h5')

Using Theano backend.


In [9]:
from pickle import load

scaler_market = load(open('../models/scaler_market.pkl', 'rb'))
scaler_no_market = load(open('../models/scaler.pkl', 'rb'))

In [10]:
stocks = []

for symbol in tqdm(symbols):
    try:
        stock = alpaca_control.polygon.historic_agg_v2(symbol, 1, 'day',
                                               _from='2018-01-01', to='2021-06-01', limit=500).df
        stock['symbol'] = symbol
        stocks.append(stock)
    except Exception as e:
        print(e)
        continue

In [11]:
stocks = pd.concat(stocks)

In [12]:
stocks.reset_index(inplace=True)

In [13]:
stocks.columns = ['date', 'open', 'high', 'low', 'close', 'volume', 'symbol']
stocks.set_index('symbol', inplace=True)
stocks.date = pd.to_datetime(stocks.date, utc=True)

In [14]:
market = alpaca_control.polygon.historic_agg_v2('SPY', 1, 'day',
                                        _from='2018-01-01', to='2021-06-01').df
market['symbol'] = 'SPY'
market.reset_index(inplace=True)
market.columns = ['date', 'open', 'high', 'low', 'close', 'volume', 'symbol']
market.set_index('symbol', inplace=True)
market.date = pd.to_datetime(market.date, utc=True)

In [15]:
open_positions_control = {position.symbol: position for position in alpaca_control.list_positions()}
open_positions_market = {position.symbol: position for position in alpaca_market.list_positions()}
open_positions_no_market = {position.symbol: position for position in alpaca_no_market.list_positions()}

In [16]:
def buy_stock(api, symbol, qty):
    try:
        api.submit_order(
            symbol=symbol,
            side='buy',
            type='market',
            qty=qty,
            time_in_force='day'
        )
    except Exception as e:
        print(f'An error occured attempting to buy {qty} shares of {symbol}', e)  

In [17]:
def sell_stock(api, symbol, qty):
    try:
        api.submit_order(
            symbol=symbol,
            side='sell',
            type='market',
            qty=qty,
            time_in_force='day'
        )
    except Exception as e:
        print(f'An error occured attempting to sell {qty} shares of {symbol}', e)

In [18]:
def predict(stock, columns, model, scaler):
    return ci.predict_today(stock, columns, model, scaler)

In [19]:
alpaca_control.cancel_all_orders()

stocks_by_symbol = stocks.groupby('symbol')

for symbol in tqdm(symbols):
    if symbol not in open_positions_control:
        last_close = stocks_by_symbol.get_group(symbol).close.iloc[-1]
        qty = 1000 // last_close
        buy_stock(alpaca_control, symbol, qty)

An error occured attempting to buy 0.0 shares of AMZN qty must be > 0
An error occured attempting to buy 0.0 shares of BKNG qty must be > 0
An error occured attempting to buy 0.0 shares of GOOG qty must be > 0
An error occured attempting to buy 0.0 shares of GOOGL qty must be > 0



In [20]:
alpaca_market.cancel_all_orders()

stocks_by_symbol = stocks.groupby('symbol')

ci.compute(market)

for symbol in tqdm(symbols):
    if symbol in stocks.index: 
        stock = stocks_by_symbol.get_group(symbol)
        ci.compute(stock)
        stock = stock.merge(market, on='date', suffixes=['', '_market'])
        prediction, last_close = predict(stock, columns_market, model_market, scaler_market)
        if (prediction >= 0.5) and (symbol not in open_positions_market):
            qty = 1000 // last_close
            buy_stock(alpaca_market, symbol, qty)
        elif (prediction < 0.5) and (symbol in open_positions_market):
            qty = open_positions_market[symbol].qty
            sell_stock(alpaca_market, symbol, qty)

In [25]:
alpaca_no_market.cancel_all_orders()

stocks_by_symbol = stocks.groupby('symbol')

for symbol in tqdm(symbols):
    if symbol in stocks.index: 
        stock = stocks_by_symbol.get_group(symbol)
        ci.compute(stock)
        prediction, last_close = predict(stock, columns_no_market, model_no_market, scaler_no_market)
        if (prediction >= 0.5) and (symbol not in open_positions_no_market):
            qty = 1000 // last_close
            buy_stock(alpaca_no_market, symbol, qty)
        elif (prediction < 0.5) and (symbol in open_positions_no_market):
            qty = open_positions_no_market[symbol].qty
            sell_stock(alapca_no_market, symbol, qty)

AAPL 0.46053404
ABBV 0.5054922
ABT 0.5054922
ACN 0.50218797
ADBE 0.46398082
AIG 0.5047265
ALL 0.5054922
AMGN 0.51267433
AMT 0.5054922
AMZN 0.5054922
AXP 0.50506943
BA 0.5395724
BAC 0.41742536
BIIB 0.5054922
BK 0.5015514
BKNG 0.5054922
BLK 0.48573357
BMY 0.4303198
BRK.B 0.5054922
C 0.4441053
CAT 0.5054922
CHTR 0.5030154
CL 0.5054922
CMCSA 0.49134445
COF 0.46591407
COP 0.4564997
COST 0.5054922
CRM 0.4685685
CSCO 0.51355493
CVS 0.5054922
CVX 0.5054922
DD 0.44868514
DHR 0.50318795
DIS 0.490082
DOW 0.46788916
DUK 0.5054922
EMR 0.47675183
EXC 0.5054922
F 0.5027878
FB 0.5054922
FDX 0.49786106
GD 0.51578456
GE 0.5054922
GILD 0.5181354
GM 0.5054922
GOOG 0.48291233
GOOGL 0.4798887
GS 0.4843163
HD 0.49583232
HON 0.5054922
IBM 0.50371885
INTC 0.46072558
JNJ 0.5054922
JPM 0.43033037
KHC 0.5209504
KMI 0.50306726
KO 0.5054922
LLY 0.5194993
LMT 0.5355057
LOW 0.48295772
MA 0.4717656
MCD 0.50378454
MDLZ 0.49225184
MDT 0.5620785
MET 0.4817457
MMM 0.47484684
MO 0.543062
MRK 0.5148444
MS 0.45294246
MSFT 0.